In [9]:
import pandas as pd
import numpy as np
from IPython.display import Image
from sklearn import tree
import pydotplus
import warnings

warnings.filterwarnings('ignore')

raw_text = pd.read_csv('/Users/junjiexie/Documents/gu校内生活/additionalData/trumptweets.csv')
# avoid the vocabulary dictionary is too heavy
raw_text = raw_text.sample(n=2000)

from datetime import datetime

raw_text.date = pd.to_datetime(raw_text.date, format="%Y-%m-%d", errors='coerce')

def beforePresident(date):
    year = str(date)[:4]
    if int(year) <= 2016:
        return 0
    else:
        return 1


raw_text['label'] = raw_text.date.apply(lambda x: beforePresident(x))

In [10]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk

english_words = set(nltk.corpus.words.words())


def first_clean(text):
    rawData = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    return rawData


def clean_not_words(input):
    input = input.split()
    return [i for i in input if i in english_words and len(i) > 2]


def lemmed_word(input):
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in input]
    return " ".join(lemmed)


sentence = raw_text['content']
first_clean_sentence = [first_clean(i) for i in sentence]
second_clean_sentence = [clean_not_words(i) for i in first_clean_sentence]
third_clean_sentence = [lemmed_word(i) for i in second_clean_sentence]

In [11]:
count = TfidfVectorizer(stop_words='english')
sentence_count = count.fit_transform(third_clean_sentence)
ColumnNames = count.get_feature_names()
DF_TF = pd.DataFrame(sentence_count.toarray(), columns=ColumnNames)
DF_TF["text_label"] = [i for i in raw_text['label']]
# DF_TF is too heavy and we should subset part of it to train
dataset = DF_TF

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset.iloc[:, :-1], dataset.iloc[:, -1:], test_size=0.3,
                                                    random_state=0)

In [12]:
dataset

,abandon,abandoned,ability,able,aboard,abortion,abroad,absolute,absolutely,absurd,...,yesterday,yoga,yogi,york,young,younger,youth,zero,zone,text_label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.60147,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [13]:
from sklearn.svm import LinearSVC
model_svm = LinearSVC(random_state=0)
model_svm.fit(X_train,y_train)
y_pred_svm = model_svm.predict(X_test)

print("Accuracy on training set: {:.3f}".format(model_svm.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(model_svm.score(X_test, y_test)))

Accuracy on training set: 0.986
Accuracy on test set: 0.822


In [14]:
model_svm_1 = LinearSVC(random_state=0, C=0.6)
model_svm_1.fit(X_train,y_train)
y_pred_svm_1 = model_svm_1.predict(X_test)

print("Accuracy on training set: {:.3f}".format(model_svm_1.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(model_svm_1.score(X_test, y_test)))

Accuracy on training set: 0.977
Accuracy on test set: 0.818


In [15]:
import plotly.figure_factory as ff
from sklearn.metrics import confusion_matrix

z = confusion_matrix(y_test, y_pred_svm_1)

label_x = ['TrumpTweetsBeforePresident', 'TrumpTweetsAfterPresident']
label_y = ['TrumpTweetsBeforePresident', 'TrumpTweetsAfterPresident']

# change each element of z to type string for annotations
z_text = [[str(y) for y in label_x] for label_x in z]

# set up figure
fig = ff.create_annotated_heatmap(z, x=label_x, y=label_y, annotation_text=z_text, colorscale='Viridis')

# add title
fig.update_layout(title_text='<i><b>Confusion matrix</b></i>',
                  )

# add custom xaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=0.5,
                        y=-0.15,
                        showarrow=False,
                        text="Predicted Value in X Axis",
                        xref="paper",
                        yref="paper"))

# add custom yaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=-0.35,
                        y=0.5,
                        showarrow=False,
                        text="Real Value in Y Axis",
                        textangle=-90,
                        xref="paper",
                        yref="paper"))

# adjust margins to make room for yaxis title
fig.update_layout(margin=dict(t=50, l=200))

# add colorbar
fig['data'][0]['showscale'] = True
fig.show()

In [16]:
from sklearn.svm import SVC
model_svm_2 = SVC(random_state=0,kernel='rbf')
model_svm_2.fit(X_train,y_train)
y_pred_svm_2 = model_svm.predict(X_test)

print("Accuracy on training set: {:.3f}".format(model_svm_2.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(model_svm_2.score(X_test, y_test)))

Accuracy on training set: 0.986
Accuracy on test set: 0.822


In [18]:
from sklearn.svm import SVC
model_svm_2_1 = SVC(random_state=0,kernel='rbf',C=0.6)
model_svm_2_1.fit(X_train,y_train)
y_pred_svm_2_1 = model_svm.predict(X_test)

print("Accuracy on training set: {:.3f}".format(model_svm_2_1.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(model_svm_2_1.score(X_test, y_test)))

Accuracy on training set: 0.858
Accuracy on test set: 0.763


In [19]:
import plotly.figure_factory as ff
from sklearn.metrics import confusion_matrix

z = confusion_matrix(y_test, y_pred_svm_2)

label_x = ['TrumpTweetsBeforePresident', 'TrumpTweetsAfterPresident']
label_y = ['TrumpTweetsBeforePresident', 'TrumpTweetsAfterPresident']

# change each element of z to type string for annotations
z_text = [[str(y) for y in label_x] for label_x in z]

# set up figure
fig = ff.create_annotated_heatmap(z, x=label_x, y=label_y, annotation_text=z_text, colorscale='Viridis')

# add title
fig.update_layout(title_text='<i><b>Confusion matrix</b></i>',
                  )

# add custom xaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=0.5,
                        y=-0.15,
                        showarrow=False,
                        text="Predicted Value in X Axis",
                        xref="paper",
                        yref="paper"))

# add custom yaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=-0.35,
                        y=0.5,
                        showarrow=False,
                        text="Real Value in Y Axis",
                        textangle=-90,
                        xref="paper",
                        yref="paper"))

# adjust margins to make room for yaxis title
fig.update_layout(margin=dict(t=50, l=200))

# add colorbar
fig['data'][0]['showscale'] = True
fig.show()

In [21]:
from sklearn.svm import SVC
model_svm_3 = SVC(random_state=0,kernel='sigmoid')
model_svm_3.fit(X_train,y_train)
y_pred_svm_3 = model_svm.predict(X_test)

print("Accuracy on training set: {:.3f}".format(model_svm_3.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(model_svm_3.score(X_test, y_test)))

Accuracy on training set: 0.901
Accuracy on test set: 0.828


In [33]:
from sklearn.svm import SVC
model_svm_3_1 = SVC(random_state=0,kernel='sigmoid',C=0.6)
model_svm_3_1.fit(X_train,y_train)
y_pred_svm_3_1 = model_svm_3_1.predict(X_test)

print("Accuracy on training set: {:.3f}".format(model_svm_3_1.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(model_svm_3_1.score(X_test, y_test)))

Accuracy on training set: 0.869
Accuracy on test set: 0.815


In [34]:
import plotly.figure_factory as ff
from sklearn.metrics import confusion_matrix

z = confusion_matrix(y_test, y_pred_svm_3_1)

label_x = ['TrumpTweetsBeforePresident', 'TrumpTweetsAfterPresident']
label_y = ['TrumpTweetsBeforePresident', 'TrumpTweetsAfterPresident']

# change each element of z to type string for annotations
z_text = [[str(y) for y in label_x] for label_x in z]

# set up figure
fig = ff.create_annotated_heatmap(z, x=label_x, y=label_y, annotation_text=z_text, colorscale='Viridis')

# add title
fig.update_layout(title_text='<i><b>Confusion matrix</b></i>',
                  )

# add custom xaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=0.5,
                        y=-0.15,
                        showarrow=False,
                        text="Predicted Value in X Axis",
                        xref="paper",
                        yref="paper"))

# add custom yaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=-0.35,
                        y=0.5,
                        showarrow=False,
                        text="Real Value in Y Axis",
                        textangle=-90,
                        xref="paper",
                        yref="paper"))

# adjust margins to make room for yaxis title
fig.update_layout(margin=dict(t=50, l=200))

# add colorbar
fig['data'][0]['showscale'] = True
fig.show()